## 1-Traitement des valeurs foncières

### A - Importation des données

Première étape, on charge les 4 fichiers .txt correspondants aux données sur les valeurs foncières de  2018 à 2021.  
On les regroupe en un seul dataframe 'df'.

In [1]:
import pandas as pd

df = pd.read_csv('./donnees/valeursfoncieres-2021.txt', sep="|", decimal=",")
df2 = pd.read_csv('./donnees/valeursfoncieres-2020.txt', sep="|", decimal=",")
df3 = pd.read_csv('./donnees/valeursfoncieres-2019.txt', sep="|", decimal=",")
df4 = pd.read_csv('./donnees/valeursfoncieres-2018.txt', sep="|", decimal=",")
df.head()


df = pd.concat([df, df2, df3, df4], axis=0, ignore_index=True)
df

/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_54912/993720581.py:3: DtypeWarning: Columns (18,23,24,26,28,30,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./donnees/valeursfoncieres-2021.txt', sep="|", decimal=",")
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_54912/993720581.py:4: DtypeWarning: Columns (18,23,24,26,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./donnees/valeursfoncieres-2020.txt', sep="|", decimal=",")
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_54912/993720581.py:5: DtypeWarning: Columns (18,23,24,26,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('./donnees/valeursfoncieres-2019.txt', sep="|", decimal=",")
/var/folders/0b/1f6rtdh943b_w3yh5c9675cw0000gn/T/ipykernel_54912/993720581.py:6: DtypeWarning: Columns (14,18,23,24,26,28,32,41) have mixed types. Specify dtype opt

: 

'df' contient 15125102 lignes & 43 colonnes.

### B - Nettoyage des données

In [ ]:
pourcentage_manquant = (df.isna().sum() / len(df)) * 100
pourcentage_manquant = pourcentage_manquant.sort_values(ascending=False)
pourcentage_manquant 

NameError: name 'df' is not defined

On supprime les ventes qui apparaissent plus d'une fois le même jour ET à la même adresse:  
-On créer un nouvelle colonne 'col_concat' avec l'adresse et la date de vente jointe.  
-On supprime les lignes ayant la même valeur dans col_concat

In [ ]:
cols_to_join = ['Date mutation', 'Type de voie', 'No voie', 'Code voie', 'Voie', 'Code postal', 'Commune']
df['col_concat'] = df[cols_to_join].apply(lambda x: ''.join(map(str,x)), axis=1)
df = df.drop_duplicates(subset='col_concat', keep=False)
df

On constate que plusieurs variables sont vides ou presques vide.  
On supprime ces variables ainsi que certaines autres qui ne seront pas exploitées dans les modèles.

In [ ]:
df = df.dropna(axis=1, how='all') 
df = df.drop(columns= {'Surface Carrez du 5eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 3eme lot', 'No Volume', '5eme lot', '4eme lot', '3eme lot', 'Surface Carrez du 2eme lot', 'Nature culture speciale', 'B/T/Q',  'Prefixe de section', '2eme lot', 'Code type local', 'No voie', 'Code voie', 'Code departement'})
df

On supprime les lignes qui n'ont pas de 'type local' renseigné car c'est une variable utilisée pour l'entrainement des modèles.

In [ ]:
df = df.dropna(subset=['Type local'])
df

On supprime les outliers concernant la valeur foncière par précaution pour ne pas perturber les modèles.

In [ ]:
Q1 = df['Valeur fonciere'].quantile(0.25)
Q3 = df['Valeur fonciere'].quantile(0.75)
IQR = Q3-Q1
df = df[(df['Valeur fonciere'] >= Q1 - 1.5 * IQR) & (df['Valeur fonciere'] <= Q3 + 1.5 * IQR)]

Étant donné que le but de nos modèle sera de prédire la valeurs foncière, correspondant donc à une vente, nous décidons de supprimer les lignes où 'Nature mutation' n'est pas 'vente'.

In [ ]:
df = df.loc[df['Nature mutation'] == 'Vente']

### C - Open Data

In [ ]:
print("Taille de notre dataframe nettoyé :", df.shape)

On a perdu ? lignes et ? colonnes en nettoyant les données.  
On va maintenant enrichir le dataframe avec de l'Open Data.

#### Taux de chômage

On a trouvé un fichier sl_etc_2023T2.xls contenant le taux de chômage par département et par année. Ça pourra nous aider dans les modèles de régression.  
Après un nettoyage du fichier, nous avons pu rajouter à notre dataframe une nouvelle variable 'Moyenne Taux Chomage' indiquant la moyenne du taux de chômage  
entre les années 2018 à 2021 par département.

In [ ]:
import pandas as pd
chomage = pd.read_excel('sl_etc_2023T2.xls',sheet_name='Département')

total_rows = len(chomage)
chomage = chomage.iloc[:total_rows - 4]
chomage

,Code,Libellé,T1_1982,T2_1982,T3_1982,T4_1982,T1_1983,T2_1983,T3_1983,T4_1983,...,T1_2021,T2_2021,T3_2021,T4_2021,T1_2022,T2_2022,T3_2022,T4_2022,T1_2023,T2_2023
0,01,AIN,3.8,3.9,4.0,4.1,4.2,4.2,4.4,4.6,...,6.2,5.9,6.0,5.6,5.5,5.5,5.5,5.3,5.3,5.4
1,02,AISNE,8.3,8.4,8.6,8.5,8.5,8.5,8.8,9.2,...,11.1,10.9,11.3,10.7,10.6,10.5,10.5,10.3,10.3,10.3
2,03,ALLIER,7.3,7.5,7.7,7.7,7.7,7.7,7.8,8.4,...,8.3,8.2,8.2,7.8,7.7,7.7,7.7,7.5,7.4,7.6
3,04,ALPES-DE-HAUTE-PROVENCE,5.5,5.6,5.8,6.0,6.1,6.1,6.3,6.6,...,9.2,9.1,8.8,8.3,8.2,8.2,8.4,8.1,7.9,7.9
4,05,HAUTES-ALPES,4.9,4.9,5.0,5.3,5.7,5.8,6.0,6.4,...,9.6,8.0,7.5,7.0,7.1,6.9,7.0,6.7,6.7,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,VAL-D'OISE,5.3,5.4,5.5,5.5,5.6,5.6,5.6,5.9,...,9.0,8.7,8.9,8.3,8.1,8.1,8.0,7.9,7.7,7.8
96,971,GUADELOUPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.3,19.5,16.4,15.5,18.1,18.5,18.3,19.3,18.2,19.5
97,972,MARTINIQUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.8,14.8,13.2,10.9,12.5,13.6,13.3,10.2,12.4,10.7
98,973,GUYANE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.3,13.5,17.1,14.2,15.5,13.3,13.5,10.9,13.9,13.8


In [ ]:
colonnes_selectionnees = chomage.filter(like='2018').columns.tolist() + \
                         chomage.filter(like='2019').columns.tolist() + \
                         chomage.filter(like='2020').columns.tolist() + \
                         chomage.filter(like='2021').columns.tolist()

nouveau_chomage = chomage[colonnes_selectionnees]
nouveau_chomage


,T1_2018,T2_2018,T3_2018,T4_2018,T1_2019,T2_2019,T3_2019,T4_2019,T1_2020,T2_2020,T3_2020,T4_2020,T1_2021,T2_2021,T3_2021,T4_2021
0,6.4,6.3,6.4,6.2,6.2,6.0,6.1,6.0,5.8,5.6,6.8,6.1,6.2,5.9,6.0,5.6
1,12.8,12.6,12.3,12.0,12.1,11.6,11.6,11.3,11.1,9.5,12.2,11.0,11.1,10.9,11.3,10.7
2,9.6,9.4,9.3,9.1,9.2,8.9,8.8,8.7,8.3,7.8,9.5,8.3,8.3,8.2,8.2,7.8
3,10.7,10.5,10.3,10.2,10.2,9.8,9.8,9.3,8.9,8.4,10.2,8.9,9.2,9.1,8.8,8.3
4,8.5,8.5,8.5,8.3,8.2,8.0,7.9,7.8,7.4,7.1,8.5,7.8,9.6,8.0,7.5,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.4,9.3,9.1,8.9,8.8,8.5,8.5,8.3,8.1,7.3,9.6,8.9,9.0,8.7,8.9,8.3
96,24.0,21.0,23.5,22.0,21.8,19.5,20.6,20.8,18.9,11.4,19.5,19.7,17.3,19.5,16.4,15.5
97,15.6,18.3,17.1,18.3,16.2,14.4,13.8,15.3,11.8,8.2,15.4,14.2,12.8,14.8,13.2,10.9
98,18.1,19.5,19.2,19.1,17.6,20.2,20.0,19.6,16.4,8.7,18.7,20.5,12.3,13.5,17.1,14.2


In [ ]:
code_postal = chomage['Code'].astype('str')
code_postal

0      01
1      02
2      03
3      04
4      05
     ... 
95     95
96    971
97    972
98    973
99    974
Name: Code, Length: 100, dtype: object

In [ ]:
moyenne_par_ligne = nouveau_chomage.mean(axis=1)
moyenne_par_ligne

0      6.10000
1     11.50625
2      8.71250
3      9.53750
4      8.03750
        ...   
95     8.72500
96    19.46250
97    14.39375
98    17.16875
99    20.10000
Length: 100, dtype: float64

In [ ]:
data = {
    'Code postal': code_postal,
    'Moyenne Taux Chomage': moyenne_par_ligne
}

moyenne_taux_chomage = pd.DataFrame(data)

print(moyenne_taux_chomage)


   Code postal  Moyenne Taux Chomage
0           01               6.10000
1           02              11.50625
2           03               8.71250
3           04               9.53750
4           05               8.03750
..         ...                   ...
95          95               8.72500
96         971              19.46250
97         972              14.39375
98         973              17.16875
99         974              20.10000

[100 rows x 2 columns]


In [ ]:
df = df.dropna(subset=['Code postal'])
# df = df.dropna(subset=['Surface terrain'])


def tronquer_valeur(valeur):
    str_valeur = str(valeur)
    if len(str_valeur) == 7:
        return int(str_valeur[:2])
    elif len(str_valeur) == 6:
        return int(str_valeur[:1])
    else: 
        return str_valeur

def ajout_0(valeur):
    str_valeur = str(valeur)
    if len(str_valeur) == 1:
        return ('0' + str(str_valeur))
    else: 
        return str_valeur
    
df['Code postal'] = df['Code postal'].astype(str)
df['Code postal'] = df['Code postal'].apply(tronquer_valeur)
df['Code postal'] = df['Code postal'].apply(ajout_0)
df['Code postal'] = df['Code postal'].astype(str)

In [ ]:
df_merged = df.merge(moyenne_taux_chomage, on='Code postal', how='left')
df_merged

,No disposition,Date mutation,Nature mutation,Valeur fonciere,Type de voie,Voie,Code postal,Commune,Code commune,Section,...,1er lot,Surface Carrez du 1er lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,col_concat,Moyenne Taux Chomage
0,1,04/01/2021,Vente,204332.0,ALL,DES ECUREUILS,01,BUELLAS,65,B,...,NaN,NaN,0,Maison,88.0,4.0,S,866.0,04/01/2021ALL7.00276DES ECUREUILS1310.0BUELLAS,6.10000
1,2,04/01/2021,Vente,226700.0,CHE,DU MOULIN DE POLAIZE,01,POLLIAT,301,AA,...,NaN,NaN,0,Maison,96.0,3.0,NaN,NaN,04/01/2021CHE173.00164DU MOULIN DE POLAIZE1310...,6.10000
2,1,08/01/2021,Vente,185000.0,RUE,DES GRANGES BONNET,01,PERONNAS,289,AD,...,NaN,NaN,0,Maison,100.0,4.0,S,703.0,08/01/2021RUE46.00161DES GRANGES BONNET1960.0P...,6.10000
3,1,07/01/2021,Vente,114500.0,RUE,DE LA MAIRIE,01,FOISSIAT,163,AB,...,NaN,NaN,0,Maison,85.0,2.0,S,87.0,07/01/2021RUE179.00110DE LA MAIRIE1340.0FOISSIAT,6.10000
4,1,08/01/2021,Vente,145000.0,IMP,DE CHAMANDRE,01,FOISSIAT,163,WC,...,NaN,NaN,0,Maison,92.0,1.0,S,2480.0,08/01/2021IMP8.00255DE CHAMANDRE1340.0FOISSIAT,6.10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765379,1,27/12/2018,Vente,1800.0,PL,DES VOSGES,75,PARIS 04,104,AO,...,109.0,NaN,1,Appartement,20.0,2.0,NaN,NaN,27/12/2018PL9.09917DES VOSGES75004.0PARIS 04,6.63125
2765380,1,28/12/2018,Vente,405000.0,RUE,BEAUTREILLIS,75,PARIS 04,104,AQ,...,16.0,33.87,2,Appartement,34.0,1.0,NaN,NaN,28/12/2018RUE13.00797BEAUTREILLIS75004.0PARIS 04,6.63125
2765381,1,26/12/2018,Vente,220000.0,RUE,DES LIONS SAINT PAUL,75,PARIS 04,104,AQ,...,126.0,NaN,2,Appartement,29.0,1.0,NaN,NaN,26/12/2018RUE14.05702DES LIONS SAINT PAUL75004...,6.63125
2765382,1,03/12/2018,Vente,383000.0,RUE,POISSONNIERE,75,PARIS 02,102,AO,...,9.0,34.78,1,Appartement,34.0,1.0,NaN,NaN,03/12/2018RUE12.07561POISSONNIERE75002.0PARIS 02,6.63125


In [1]:
# faire un graph du taux de chomage par departement
chomage_par_dep = df.groupby('Code postal')['Moyenne Taux Chomage'].mean()
# Créez un histogramme
plt.bar(chomage_par_dep.index, chomage_par_dep.values)

# Étiquetez les axes et le titre
plt.xlabel('Code postal')
plt.ylabel('Moyenne Taux Chomage')
plt.title('Histogramme du taux moyen de chômage par departement')

# Affichez l'histogramme
plt.show()

#### Moyenne prix/m2/departement

On a créer deux nouvelles variables 'prix_par_m2' et 'moyenne_prix_par_m2_par_code_postal' calculée directement à partir des vraiables 'Valeur fonciere', 'Surface reelle bati' et 'Code postal' de notre dataframe. Ce seront des bons indicateurs pour nos modèles.

In [ ]:
df['prix_par_m2'] = df['Valeur fonciere'] / df['Surface reelle bati']

pd.to_numeric(df['Valeur fonciere'], errors='coerce')

grouped_values = df.groupby('Code postal')['prix_par_m2'].mean()

df['moyenne_prix_par_m2_par_code_postal'] = df['Code postal'].map(grouped_values)  # Ajoute la moyenne par 'Code postal'
df

### D - Exploration des données nettoyées

Voici quelques graphs permettant de se faire une idée du dataframe df

In [ ]:
df.shape()

In [ ]:
import plotly.express as px

In [ ]:
fig1 = px.histogram(df, x='Type local', y='Valeur fonciere', histfunc='avg', title='Moyenne des prix des biens par type de local', color="Type local")

In [ ]:
fig2 = px.histogram(df, x='Type local', y='Surface terrain', histfunc='avg', title='Moyenne des surfaces des biens par type de local', color="Type local")

In [ ]:
fig3 = px.scatter(df, x='Commune', y="Valeur fonciere", title='Prix des biens dans les communes', color="Type local")

In [ ]:
fig4 = px.histogram(df, x="Mois", y='Valeur fonciere', title='Nombre de Ventes par mois', histfunc='count')

In [ ]:
fig5 = px.histogram(df, x="Mois", y='Valeur fonciere', title='Moyennes des prix de ventes par mois', histfunc='avg')

In [ ]:
fig6 = px.scatter(df, x='Surface reelle bati', y="Valeur fonciere", title='Prix des biens avec leur superficie')
fig6.update_xaxes(range=[10, 600]) 

In [ ]:
fig7 = px.histogram(df, x="Surface reelle bati", y="Valeur fonciere", color="Type local", marginal="rug", hover_data=df.columns)
fig7.update_xaxes(range=[10, 500])

## 2 - Modèle de Classification sur la variable 'Type local'

Le but final du projet est de prédire la valeur foncière d'un bien. Une étape intermédiaire pour y parvenir est de créer un modèle pour prédire la variable 'type local', car c'est une variable explicative très importante qui nous servira ensuite pour les modèle de regression sur la valeur foncière.  
Pour prédire le 'type local' nous utilisons un arbre de décision.

In [ ]:
import pickle
import pandas as pd

#### Création des groupes train et test

In [ ]:
# Sélectionner uniquement les lignes où 'Type local' est renseigné
df_filtered = df[df['Type local'].notna()]

In [ ]:
# on prend quelques variables explicatives

X = df_filtered[["Surface reelle bati", "Surface terrain", "Nombre pieces principales", "Nombre de lots","Code commune" ]]

#on nomme la variale cible

Y= df_filtered["Type local"]

In [ ]:
pip install scikit-learn


  Using cached scikit_learn-1.3.1-cp311-cp311-win_amd64.whl (9.2 MB)
  Using cached scipy-1.11.3-cp311-cp311-win_amd64.whl (44.1 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# on scinde les données en echantillon test et train
#70% des données pour l’apprentissage
#30% des données pour l'échantillon test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size = 0.30,
                                                    stratify = Y,
                                                    random_state = 42)

On va gerer les valeurs manquantes dans l'echantillon d'apprentissage

In [ ]:
X_train['Nombre pieces principales'].fillna(X_train['Nombre pieces principales'].mean(), inplace=True)
X_test['Nombre pieces principales'].fillna(X_test['Nombre pieces principales'].mean(), inplace=True)
X_train['Surface reelle bati'].fillna(X_train['Surface reelle bati'].mean(), inplace=True)
X_test['Surface reelle bati'].fillna(X_test['Surface reelle bati'].mean(), inplace=True)
X_train["Surface terrain"].fillna(X_train["Surface terrain"].mean(), inplace=True)
X_test["Surface terrain"].fillna(X_test["Surface terrain"].mean(), inplace=True)
X_train["Nombre de lots"].fillna(X_train["Nombre de lots"].mean(), inplace=True)
X_test["Nombre de lots"].fillna(X_test["Nombre de lots"].mean(), inplace=True)
X_train["Code commune"].fillna(X_train["Code commune"].mean(), inplace=True)
X_test["Code commune"].fillna(X_test["Code commune"].mean(), inplace=True)


#### Apprentissage

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

In [ ]:
model_1 = DecisionTreeClassifier(max_depth=3 , min_samples_leaf=10, min_samples_split=20)
model_1 = model_1.fit(X_train,y_train)

#### Prediction

In [ ]:
y_pred = model_1.predict(X_test)
y_pred[0:10]

array(['Maison', 'Local industriel. commercial ou assimilé', 'Maison',
       'Maison', 'Dépendance', 'Maison', 'Maison', 'Maison',
       'Appartement', 'Maison'], dtype=object)

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score

print('recall : ' + str(recall_score(y_test,y_pred,average='macro')))
print('precision : ' + str(precision_score(y_test,y_pred, average='macro')))
print('f1_score : ' + str(f1_score(y_test,y_pred, average='macro')))


recall : 0.9730989707955009
precision : 0.9681546865145049
f1_score : 0.9700326574267171


#### Optimisation des parametres de l'arbre

On va utiliser un greedsearch pour améliorer les performances du modèle

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

In [ ]:
parameters = {'max_depth' : [1,2,3,4,5,6] ,
              'min_samples_leaf' : [10,20,30,40],
              'min_samples_split' : [50,100,200]}

dtc = DecisionTreeClassifier()
dtc_model_1 = GridSearchCV(dtc, parameters, scoring = 'accuracy' ,
                   cv = ShuffleSplit(test_size=0.30, n_splits=1, random_state=0))
dtc_model_1.fit(X_train, y_train )

print("Voici les paramètres du meilleure modèle : " + str(dtc_model_1.best_estimator_))
print("Voici le score "  + str(dtc_model_1.scorer_) + " du meilleure modèle : " + str(dtc_model_1.best_score_))

Voici les paramètres du meilleure modèle : DecisionTreeClassifier(max_depth=6, min_samples_leaf=10, min_samples_split=100)
Voici le score make_scorer(accuracy_score) du meilleure modèle : 0.9731413574039098


In [ ]:
y_pred_dtc = dtc_model_1.predict(X_test)

In [ ]:
print('recall : ' + str(recall_score(y_test,y_pred_dtc,average='macro')))
print('precision : ' + str(precision_score(y_test,y_pred_dtc, average='macro')))
print('f1_score : ' + str(f1_score(y_test,y_pred_dtc, average='macro')))

recall : 0.9776428794967764
precision : 0.9710341328596991
f1_score : 0.9741343893976641


Les performances du modèle sont très bonnes.

## 3 - Reggresion sur la variable 'Valeur fonciere' 

In [12]:
# librairie utilisées
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


Maintenant que nous pouvons prédire la variable des 'Type local' nous pouvons passer à l'étape final qui est de trouver le meilleur modèle de régression afin de prédire les meilleures 'Valeur foncière'. Pour cette étape nous avons étudié pas mal d'options. Tout d'abord le choix des variables explicatives. Après pas mal de test, nous avons trouvé que les variables les plus pertinentes et qui renvoyaient les meilleurs résultats sont : 'Type local', 'Nombre pieces principales', 'Surface reelle bati', 'Surface terrain', 'Nombre de lots', 'Moyenne Taux Chomage', 'prix_par_m2', 'moyenne_prix_par_m2_par_code_postal'. A cela nous avons ajouté des données OpenData qui sont le prix par m2 de la vente et la moyenne du prix par m2 dans le département.   

In [4]:
# pickle de notre dataframe retravaillé, contenant les données open source
with open('dataframe2.pkl', 'rb') as file:
    df = pickle.load(file)
df.head(3)

,No disposition,Date mutation,Nature mutation,Valeur fonciere,Type de voie,Voie,Code postal,Commune,Code commune,Section,...,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,col_concat,Moyenne Taux Chomage,prix_par_m2,moyenne_prix_par_m2_par_code_postal
0,1,04/01/2021,Vente,204332.0,ALL,DES ECUREUILS,01,BUELLAS,65,B,...,0,Maison,88.0,4.0,S,866.0,04/01/2021ALL7.00276DES ECUREUILS1310.0BUELLAS,6.1,2321.954545,184483.787842
1,2,04/01/2021,Vente,226700.0,CHE,DU MOULIN DE POLAIZE,01,POLLIAT,301,AA,...,0,Maison,96.0,3.0,NaN,NaN,04/01/2021CHE173.00164DU MOULIN DE POLAIZE1310...,6.1,2361.458333,184483.787842
2,1,08/01/2021,Vente,185000.0,RUE,DES GRANGES BONNET,01,PERONNAS,289,AD,...,0,Maison,100.0,4.0,S,703.0,08/01/2021RUE46.00161DES GRANGES BONNET1960.0P...,6.1,1850.000000,184483.787842


### A - Echantillonage

In [7]:
variables_explicatives = ['Type local', 'Nombre pieces principales', 'Surface reelle bati', 
                          'Surface terrain', 'Nombre de lots', 'Moyenne Taux Chomage', 'prix_par_m2', 
                          'moyenne_prix_par_m2_par_code_postal']


X = df[variables_explicatives]
X = pd.get_dummies(data=X, columns=['Type local'])
X.head(3)

,Nombre pieces principales,Surface reelle bati,Surface terrain,Nombre de lots,Moyenne Taux Chomage,prix_par_m2,moyenne_prix_par_m2_par_code_postal,Type local_Appartement,Type local_Dépendance,Type local_Local industriel. commercial ou assimilé,Type local_Maison
0,4.0,88.0,866.0,0,6.1,2321.954545,184483.787842,False,False,False,True
1,3.0,96.0,NaN,0,6.1,2361.458333,184483.787842,False,False,False,True
2,4.0,100.0,703.0,0,6.1,1850.000000,184483.787842,False,False,False,True


In [8]:
# notre target
Y = df['Valeur fonciere']
Y

0          204332.0
1          226700.0
2          185000.0
3          114500.0
4          145000.0
             ...   
2765379      1800.0
2765380    405000.0
2765381    220000.0
2765382    383000.0
2765383     45000.0
Name: Valeur fonciere, Length: 2717931, dtype: float64

Pour les données manquantes, nous les avons remplacés par la moyenne de la variable. On a aussi dû traiter que des variables étaient 'inf' (venant de notre calcul du prix par m2). Ces valeurs infinies ont été remplacée par la moyenne du prix au m2 dans son département. 

In [15]:
# on remplace les NA dans les colonnes par leurs moyennes
X['Nombre pieces principales'].fillna(X['Nombre pieces principales'].mean(), inplace=True)
X['Surface reelle bati'].fillna(X['Surface reelle bati'].mean(), inplace=True)
X['Surface terrain'].fillna(X['Surface terrain'].mean(), inplace=True)
X['Nombre de lots'].fillna(X['Nombre de lots'].mean(), inplace=True)
X['Moyenne Taux Chomage'].fillna(X['Moyenne Taux Chomage'].mean(), inplace=True)
X['prix_par_m2'].fillna(X['prix_par_m2'].mean(), inplace=True)

# Remplacer les lignes ou 'prix_par_m2' est infini
masque_infini = np.isinf(X['prix_par_m2'])
# Remplacez les valeurs infinies par la valeur de 'moyenne_prix_par_m2_par_code_postal'
X.loc[masque_infini, 'prix_par_m2'] = X.loc[masque_infini, 'moyenne_prix_par_m2_par_code_postal']

In [16]:
#70% des données pour l’apprentissage
#30% des données pour l'échantillon test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

Pour ne pas donner plus d'importance aux variables explicatives à forte variance, il est essentiel de centrer et réduire les données en amont. On centre et réduit également afin de les ramener à la même échelle.

In [17]:
# Appliquez la standardisation aux données de X
scaler = StandardScaler()
X_train_CR = scaler.fit_transform(X_train)
X_test_CR = scaler.fit_transform(X_test)
pd.DataFrame(X_train_CR, columns=X_test.columns).head(3)

,Nombre pieces principales,Surface reelle bati,Surface terrain,Nombre de lots,Moyenne Taux Chomage,prix_par_m2,moyenne_prix_par_m2_par_code_postal,Type local_Appartement,Type local_Dépendance,Type local_Local industriel. commercial ou assimilé,Type local_Maison
0,-1.487268,-0.312836,-0.384831,-0.696888,-0.862184,2.763116,0.857299,-0.604256,2.493000,-0.225365,-1.095766
1,0.056106,-0.038217,-0.243456,-0.696888,2.089573,-0.399679,-0.042272,-0.604256,-0.401123,-0.225365,0.912604
2,-0.458352,-0.155911,-0.000321,1.576243,-0.979162,-0.257236,1.615838,1.654927,-0.401123,-0.225365,-1.095766


### B - Apprentissage

Pour l'apprentissage de notre modèle nous en avons testé des différents : Régression linéaire multiple, Ridge, Lasso, ElasticNet, Arbre de décision ou encore RandomForest. 

Pour maximiser les performances de nos modèles, nous avons effectué des GridSearch sur chacun d'eux afin de récupérer les meilleurs hyper-paramètres possibles. Lasso, ElasticNet et RandomForest prenaient un temps considérable d'éxecution (plus de 2h), et nous avons donc pas testé jusqu'au bout, mais lors de test manuel nous pouvions voir qu'ils ne seraient pas les plus satisfaisant. Tout comme la régression linéaire qui n'a pas rendu des très bon scores. Ci-dessous je vous montre 2 exemples de modèles qui nous ont parus performant : **Ridge** et **Arbre de Décision**. 

#### Régression Ridge

In [18]:
parameters = {'alpha': np.arange(start = 0, stop = 10, step = 0.5)}

ridge_model = Ridge()
grid_ridge = GridSearchCV(ridge_model, parameters, scoring = 'neg_mean_squared_error')
grid_ridge.fit(pd.DataFrame(X_train_CR, columns=X_train.columns), y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5])},
             scoring='neg_mean_squared_error')

In [22]:
print("Meilleur paramètre :", grid_ridge.best_params_)
print("Meilleur score :", grid_ridge.best_score_)
y_pred = grid_ridge.best_estimator_.predict(pd.DataFrame(X_test_CR, columns=X_train.columns))
print("RMSE : " + str(mean_squared_error(y_test, y_pred, squared= False)))

Meilleur paramètre : {'alpha': 9.5}
Meilleur score : -7927944888.914922
RMSE : 88917.85600111628


In [25]:
coef = pd.DataFrame(grid_ridge.best_estimator_.coef_ ,
                    index = X_train.columns, columns=['Coef'])
coef.loc['Constante'] = grid_ridge.best_estimator_.intercept_
coef

,Coef
Nombre pieces principales,44515.754832
Surface reelle bati,4775.763449
Surface terrain,2773.626653
Nombre de lots,1282.645568
Moyenne Taux Chomage,231.720966
prix_par_m2,-14954.952431
moyenne_prix_par_m2_par_code_postal,51728.111576
Type local_Appartement,-12109.164103
Type local_Dépendance,11020.874450
Type local_Local industriel. commercial ou assimilé,10991.531461


On peut noter ici que les variables les plus significatives sont 'Nombre pieces principales' et 'moyenne_prix_par_m2_par_code_postal', tandis que le 'prix_par_m2' et le 'type_local' ont leur importance aussi dans le modèle.

#### Arbre de décision

In [26]:
tree_regressor = DecisionTreeRegressor()
param_grid = {
    'max_depth': [None, 10, 20, 30],  # Profondeur maximale de l'arbre
    'min_samples_split': [2, 5, 10],  # Nombre minimal d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [1, 2, 4]  # Nombre minimal d'échantillons requis dans une feuille
}
grid_tree_regressor = GridSearchCV(tree_regressor, param_grid=param_grid, scoring="neg_mean_squared_error")
grid_tree_regressor.fit(pd.DataFrame(X_train_CR, columns=X_train.columns), y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error')

In [27]:
print("Meilleur paramètre :", grid_tree_regressor.best_params_)
print("Meilleur score :", grid_tree_regressor.best_score_)

y_pred = grid_tree_regressor.predict(pd.DataFrame(X_test_CR, columns=X_test.columns))
print("RMSE : " + str(mean_squared_error(y_test, y_pred, squared= False)))

Meilleur paramètre : {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10}
Meilleur score : -1411872005.56255
RMSE : 40351.40670365985


Les résultats de ces deux modèles furent les plus satisafaisants. Nous sommes donc partis sur la régression avec l'arbre de décision pour la prédiction des valeurs foncières sur le jeu de données Kaggle.

## 4 - Interface Dash

Pour l'interface Dash, nous avons décider de faire 3 onglets. Un onglet statistiques avec les graphiques que nous avons effectués précédement. Un onglet Cartographie, pour visualisre nos données sur le territoire Français. Et un onglet Prédictions pour prédire/ Estimer un bien immobilier.
L'interface demande un certain temps de chargement des données, c'est pour cela que nous avons limités les donnnées à 8000 lignes. Avant cela, nous avons évidemnt mélanger la datframe d'origine.

### A - Statistiques

Pour les statisques, nous avons récupérés les graphiques pour étudier nos données. Deux Filtres ont été créés sur cette page: 
* un filtre sur l'année des données
* un filtre sur le type local des donnés

Ces filtres sont reliés aux graphiques et sont modifiés une fois les filtres selectionnés.


![image.png](DASH1.png)

Tous les graphiques sont exportables sous format png à l'aide de plotly.express. Comme on peux le voir sur la photo ci dessous, le graphique en bas à droite est exportable avec le pictogramme de plotly.express.

![image.png](Dash2.png)

### B - Cartographie 

La Cartographie a été effectuée dans un premier temps sur les données initiales pour afficher le prix du mètre carré par département. Chaque Bien immobilier a eu son prix au mètres carré puis ce prix a été regroupé par département avec une moyenne.

Ensuite, pour notre apprentissage des données Open Data sur le taux de chômage par départements a été ajouté. Il nous semblais pertinent d'effectuer une cratographie sur ce taux de chômage. 

Un histogramme a été également pertinent pour mieux voir quels sont les départements avec les prix au mètres carré le plus/le moins cher.

![image.png](Dash3.png)

### C - Prédictions

Pour la partie des prédictions, un bien est estimé à partir de plusieurs champs que l'utilisateur peut remplir. Si l'utilisateur ne remplit pas ou ne connait pas le type local de son bien, nous utilisons la classsification que nous avons mis en place afin de prédire le type du bien.

Par la suite le prix du bien est prédit à l'aide du meilleur modèle de régression que nous avons effectué. 

Des règles sur ces champs sont évidements présentes. Si la surface réelle bati n'est pas rempli par exemple, nous demandons à l'utilisateur de remplir correctement les champs.

#### ??? AJOUTER IMAGE  ??? ###